In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io
from scipy.stats import ttest_ind
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import chardet


# Step 1: Ask at least two questions
# Predictive Question: Given a parent’s level of education, how likely are they going to have exam prep?
# Exploratory Question: Whether students who had exam prep have higher scores in our sample data?

# Step 2: Collect the data
# Provide the data source
data_source = "https://www.kaggle.com/datasets/spscientist/students-performance-in-exams"
# How was the data collected?
data_collection_source = "http://roycekimmons.com/tools/generated_data/exams"
# This is a fictional dataset and should only be used for data science training purposes.

# Reads data file from Google Drive
file_id = "1boOd8BVffLE914tY4n43Msyrvpx8LygS"
url = f"https://drive.google.com/uc?id={file_id}"
download = requests.get(url).content

# Detect the file encoding using chardet
detected_encoding = chardet.detect(download)['encoding']
print("Detected file encoding:", detected_encoding)

# Read the data using the detected encoding
df = pd.read_csv(io.StringIO(download.decode(detected_encoding)))

# Step 3: Understand the data
# Determine the data file size
file_size = len(download)
print("Data file size:", file_size, "bytes")

# Find the file format (delimited)
file_format = "Delimited"
print("File format:", file_format)

# Determine the file encoding (using chardet)
file_encoding = detected_encoding
print("File encoding:", file_encoding)

# What is the granularity of the data? (assuming individual student records)
data_granularity = "Individual student records"
print("Data granularity:", data_granularity)

# Data Quality Check: Are there any outliers? Are there any missing or invalid entries?
# Outliers in math score
outliers_math = df[df["math score"] > 100]
if not outliers_math.empty:
    print("There are outliers in the math score column.")
    # Data cleaning step: Replace outliers with the maximum valid score of 100
    df.loc[df["math score"] > 100, "math score"] = 100

# Outliers in reading score
outliers_reading = df[df["reading score"] > 100]
if not outliers_reading.empty:
    print("There are outliers in the reading score column.")
    # Data cleaning step: Replace outliers with the maximum valid score of 100
    df.loc[df["reading score"] > 100, "reading score"] = 100

# Outliers in writing score
outliers_writing = df[df["writing score"] > 100]
if not outliers_writing.empty:
    print("There are outliers in the writing score column.")
    # Data cleaning step: Replace outliers with the maximum valid score of 100
    df.loc[df["writing score"] > 100, "writing score"] = 100

# Missing or invalid entries
missing_values = df.isnull().sum()
if missing_values.any():
    print("There are missing or invalid entries in the data set.")
    # Data cleaning step: Handle missing values in "test preparation course" column with the most frequent value (mode)
    mode_prep_course = df["test preparation course"].mode().iloc[0]
    df["test preparation course"].fillna(mode_prep_course, inplace=True)

# Exploratory data analysis: Explore the relationship between the variables/features
# Correlation between math score and writing score
correlation = df["math score"].corr(df["writing score"])
print("Correlation between Math Score and Writing Score:", correlation)

# Visualization: Identify the feature type and use appropriate plots to visualize the relationship among variables
# Scatter plot of math score and writing score
plt.scatter(df["math score"], df["writing score"])
plt.title("Math Score vs Writing Score")
plt.xlabel("Math Score")
plt.ylabel("Writing Score")
plt.show()

# Bar plot of exam preparation and average scores
avg_scores = df.groupby("test preparation course")[["math score", "reading score", "writing score"]].mean()
avg_scores.plot(kind="bar")
plt.title("Average Scores by Exam Preparation")
plt.xlabel("Exam Preparation")
plt.ylabel("Average Score")
plt.show()

# Inferential analysis: Compare scores between students who completed the test preparation course and those who did not
completed_prep = df[df["test preparation course"] == "completed"]["math score"]
not_completed_prep = df[df["test preparation course"] == "none"]["math score"]

# Perform t-test
t_stat, p_value = ttest_ind(completed_prep, not_completed_prep)

# Determine the significance level
significance_level = 0.05

# Make a decision based on the p-value
if p_value < significance_level:
    print("Students who completed the test preparation course have a significantly higher score.")
else:
    print("There is not enough evidence to conclude a significant difference in scores.")

# Exploratory Question: How many people scored higher than 85 in math?
num_high_scores = len(df[df["math score"] > 85])
print("Number of people scoring higher than 85 in math:", num_high_scores)

# Inferential analysis for reading and writing scores: Compare scores between students who completed the test preparation course and those who did not
completed_prep_reading = df[df["test preparation course"] == "completed"]["reading score"]
not_completed_prep_reading = df[df["test preparation course"] == "none"]["reading score"]

completed_prep_writing = df[df["test preparation course"] == "completed"]["writing score"]
not_completed_prep_writing = df[df["test preparation course"] == "none"]["writing score"]

# Perform t-tests for reading and writing scores
t_stat_reading, p_value_reading = ttest_ind(completed_prep_reading, not_completed_prep_reading)
t_stat_writing, p_value_writing = ttest_ind(completed_prep_writing, not_completed_prep_writing)

# Determine the significance level for reading and writing scores
significance_level = 0.05

# Make a decision based on the p-values for reading and writing scores
if p_value_reading < significance_level:
    print("Students who completed the test preparation course have a significantly higher reading score.")
else:
    print("There is not enough evidence to conclude a significant difference in reading scores.")

if p_value_writing < significance_level:
    print("Students who completed the test preparation course have a significantly higher writing score.")
else:
    print("There is not enough evidence to conclude a significant difference in writing scores.")


# Build the linear regression model
X = df[["math score"]]  # Use math score as the feature (independent variable)
y = df["writing score"]  # Use writing score as the target (dependent variable)

## Exploratory Question: Given a parent’s level of education, how likely are they going to have exam prep?
education_categories = ['some high school', 'high school', 'some college', "associate's degree", "bachelor's degree", "master's degree"]
ohe = OneHotEncoder()
ohe.fit_transform(df[['test preparation course']])[:5]
oe = OrdinalEncoder(categories = [education_categories])
oe.fit_transform(df[['parental level of education']])[:5]
df
A = df.drop('parental level of education', axis = 1)
B= df['parental level of education']

column_transform = make_column_transformer(
    (ohe,  df),
    (oe, df))

column_transform.fit_transform(df[['test preparation course']])[:5]

#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=42)

#lm = GradientBoostingRegressor()
#lm_pipeline = make_pipeline(column_transform, lm)
#lm_pipeline.fit(X_train, Y_train)
#lm_predictions = lm_pipeline.predict(X_test)

#lm_predictions = lm_pipeline.predict(a_test)
#lm_mae = mean_absolute_error(lm_predictions, Y_test)
#lm_rmse = np.sqt(mean_squared_error(lm_predictions, Y_test))
#print(df['parental level of education'].head())


# Create an instance of the Linear Regression model
model = LinearRegression()

# Fit the model to the data
model.fit(X, y)

# Make predictions using the model
predictions = model.predict(X)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y, predictions)
print("Mean Squared Error (MSE):", mse)

# Interesting Findings
print("Interesting Findings:")
print("- The scatter plot shows a positive correlation between math score and writing score. Higher math scores tend to correspond to higher writing scores.")
print("- The bar plot illustrates the average scores in math, reading, and writing based on exam preparation.")
print("- Students who completed the test preparation course have a significantly higher score (t-test).")
print("- The number of people scoring higher than 85 in math is:", num_high_scores)
print("- Students who completed the test preparation course have a significantly higher reading score (t-test).")
print("- Students who completed the test preparation course have a significantly higher writing score (t-test).")



Detected file encoding: ascii
Data file size: 1103 bytes
File format: Delimited
File encoding: ascii
Data granularity: Individual student records


KeyError: ignored